# Test computational cost of Stochastic Programming (Scenario Optimisation)

Investigate computational complexity and runtime of scnearios optimisation for system design problem, where each scenario uses a realisation of uncertain battery efficiencies.

In [ ]:
import os
import csv
import time
import numpy as np

from citylearn.citylearn import CityLearnEnv
from linmodel import LinProgModel
from schema_builder import build_schema
from sys_eval import evaluate_system

In [ ]:
# set up test parameters
dataset_dir = os.path.join('A37_example_validate') # dataset directory
schema_path = os.path.join('data', dataset_dir, 'schema.json')
opex_factor = 10
pricing_dict = {'carbon':5e-1,'battery':1e3,'solar':2e3}

In [ ]:
# Set up base parameters of system.
ids = [5,11,14,16,24,29]

base_kwargs = {
    'output_dir_path': os.path.join('data','A37_example_validate'),
    'building_names': ['UCam_Building_%s'%id for id in ids],
    'battery_energy_capacities': [3127.0,2736.0,2746.0,2448.0,4788.0,4565.0], # from Annex 37
    'battery_power_capacities': [391.0,342.0,343.0,306.0,598.0,571.0], # from Annex 37
    'battery_efficiencies': None,
    'pv_power_capacities': [178.0,41.0,57.0,120.0,1349.0,257.0], # from Annex 37
    'load_data_paths': ['UCam_Building_%s.csv'%id for id in ids],
    'weather_data_path': 'weather.csv',
    'carbon_intensity_data_path': 'carbon_intensity.csv',
    'pricing_data_path': 'pricing.csv',
    'schema_name': 'schema_temp'
}

In [ ]:
# Set up probabilistic model of effiencies and take draws
n_draws = 100
mu = 0.9
sigma = 0.025
eta_samples = np.random.normal(loc=mu,scale=sigma,size=(n_draws,len(ids)))

In [ ]:
max_scenarios = 25
runtimes = {}

for M in range(1,max_scenarios):
    print(f"# Scenarios: {M}")

    etas = eta_samples[:M] # get M samples

    for m in range(M):
        # Build schema.
        base_kwargs.update({
                'battery_efficiencies': etas[m]
            })
        schema_path = build_schema(**base_kwargs)

        # Initialise CityLearn environment object.
        env = CityLearnEnv(schema=schema_path)

        if m == 0: # initialise lp object
            lp = LinProgModel(env=env)
        else:
            lp.add_env(env=env)

    lp.set_time_data_from_envs()
    lp.generate_LP(clip_level='b',design=True,pricing_dict=pricing_dict,opex_factor=opex_factor)
    lp.set_LP_parameters()

    start = time.time()
    results = lp.solve_LP(verbose=False,ignore_dpp=True)
    end = time.time()

    runtimes.update({M: end-start})
    print(f"Solve time: {round(runtimes[-1],1)}s")

In [1]:
# Plot computational time results

In [2]:
# Compute MC estimate of true system cost

In [3]:
# Plot convergence of MC estimate